In [11]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

Macophieu = "GOOG"
prop = 'Close'
model = tf.keras.models.load_model('lstm_stock_model.keras')

# 🟢 Bước 2: Tải dữ liệu từ Yahoo Finance
ticker = "GOOG"  # Thay đổi mã cổ phiếu nếu cần
n_past_days = 60  # Sử dụng 60 ngày gần nhất để dự đoán
future_days = 7   # Dự đoán 3 ngày tiếp theo

# Lấy dữ liệu từ Yahoo Finance
today = pd.to_datetime("2025-03-20")  # Ngày hiện tại (có thể thay đổi)
start_date = today - pd.Timedelta(days=n_past_days + 40)  # Lấy thêm để đảm bảo đủ dữ liệu
df = yf.download(ticker, start=start_date, end=today)

# 🟢 Bước 3: Chuẩn bị dữ liệu đầu vào
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices = df["Close"].values.reshape(-1, 1)
close_prices_scaled = scaler.fit_transform(close_prices)

[*********************100%***********************]  1 of 1 completed


In [13]:
x_input = close_prices_scaled[-n_past_days:].reshape(1, n_past_days, 1)

In [17]:
future_predictions = []
for i in range(future_days):
    next_day_pred = model.predict(x_input)
    next_day_pred_real = scaler.inverse_transform(next_day_pred)[0][0]
    future_predictions.append(next_day_pred_real)

    # Cập nhật dữ liệu đầu vào
    # x_input = np.append(x_input[:, 1:, :], next_day_pred.reshape(1, 1, 1), axis=1)
    x_input = np.array(x_input).reshape(1, timesteps, features)


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 1 and 9 for '{{node sequential_1/lstm_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/lstm_1/strided_slice_1, sequential_1/lstm_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [1,1], [9,320].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(1, 1), dtype=float32)
  • states=('tf.Tensor(shape=(1, 80), dtype=float32)', 'tf.Tensor(shape=(1, 80), dtype=float32)')
  • training=False

In [ ]:
for i in range(future_days):
    future_date = today + pd.Timedelta(days=i+1)
    print(f"Dự đoán giá {ticker} ngày {future_date.strftime('%d/%m/%Y')}: {future_predictions[i]:.2f}")

In [ ]:
dates = [today + pd.Timedelta(days=i+1) for i in range(future_days)]
plt.plot(dates, future_predictions, marker='o', linestyle='dashed', color='b', label="Dự đoán")
plt.xlabel("Ngày")
plt.ylabel("Giá dự đoán")
plt.title(f"Dự đoán giá {ticker} trong 7 ngày tới")
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.show()